In [33]:
# Mến chào cô!

# Em tên: Trần Huỳnh Chí Nguyên.
# MSSV: 23110136.

# Em xin phép nộp bài tập môn Trí tuệ Nhân tạo ạ!

# 💡
    # Em đặt Scale màn hình là 100% nhằm tránh bị cắt xén giao diện, nên có thể bị khuất nếu thiết bị đang có tỉ lệ khác 100% thưa cô.
    # Các thuật toán em đêu làm đủ và đặt trong Combobox (nếu bị khuất cô có thể trượt xuống để xem thêm ạ).
    # Các bài tập em đều tích lũy đầy đủ và sửa bổ xung liên tục trong file này, hy vọng cô xem qua và chăm chế các lỗi sai từ các bài trước của em ạ.

# Em xin đính kèm link gitub: 

# Em mến chúc cô sức khỏe!

import tkinter as tk
from tkinter import ttk 
import copy
from collections import deque  
from queue import PriorityQueue
import heapq
import random, math
import re
import collections

cars_positions = []
canvas_right_global = None
algorithm_frame = None
state_frame = None
algorithm_combobox = None
selected = ""
timer_elapse = 0
timer_running = False
timer_label = None
CELL_SIZE = 45

original_state = [
    [0,0,0,0],
    [0,0,0,0],
    [0,0,0,0],
    [0,0,0,0]
]

goal_state = [
    [0,0,0,1],
    [0,1,0,0],
    [0,0,1,0],
    [1,0,0,0]
]
BOARD_SIZE = len(original_state)

def update_timer():
    global timer_elapse, timer_running, timer_label
    if timer_running:
        timer_elapse+=1
        timer_label.config(text=f"Thời gian: {timer_elapse}s")
        timer_label.after(1000,update_timer)

def reset_game():
    global timer_elapse, timer_running, cars_positions, selected
    timer_elapse = 0
    timer_running = False
    cars_positions = copy.deepcopy(original_state)  
    timer_label.config(text="Thời gian: 0 s")
    draw_board(canvas_right_global, with_cars=False)
    selected = ""
    show_total_state("⚠ Vui lòng chọn 1 thuật toán trong các\n"\
                        "nhóm thuật toán: Local Search để xem\n"\
                        "được lịch sử các trạng thái đã đi qua ⚠")

def BFS():
    node = tuple(tuple(row) for row in original_state)  
    if node == tuple(tuple(row) for row in goal_state):  
        return goal_state

    frontier = deque([node]) 
    explored = set()          

    while frontier:
        node = frontier.popleft()   
        explored.add(node)

        a = [list(row) for row in node]  
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                b = [list(row) for row in a]
                if b[i][j] == 1:
                    continue
                b[i][j] = 1

                valid = True
                for r in range(BOARD_SIZE):
                    if sum(b[r]) > 1:
                        valid = False
                        break
                if valid:
                    for c2 in range(BOARD_SIZE):
                        if sum(b[r][c2] for r in range(BOARD_SIZE)) > 1:
                            valid = False
                            break

                if valid:
                    child = tuple(tuple(row) for row in b)
                    if child not in explored and all(child != n for n in frontier):
                        if child == tuple(tuple(row) for row in goal_state):
                            return [list(row) for row in child]
                        frontier.append(child)
    return None

def DFS():
    node = tuple(tuple(row) for row in original_state)  
    if node == tuple(tuple(row) for row in goal_state):  
        return goal_state

    stack = [node]        
    visited = {node}    

    while stack:
        node = stack.pop()    
        a = [list(row) for row in node]

        if a == goal_state:
            return a

        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                b = [list(row) for row in a]
                if b[i][j] == 1:
                    continue
                b[i][j] = 1

                valid = True
                for r in range(BOARD_SIZE):
                    if sum(b[r]) > 1:
                        valid = False
                        break
                if valid:
                    for c2 in range(BOARD_SIZE):
                        if sum(b[r][c2] for r in range(BOARD_SIZE)) > 1:
                            valid = False
                            break

                if valid:
                    state = tuple(tuple(row) for row in b)
                    if state not in visited:
                        visited.add(state)
                        stack.append(state)  
    return None
    
def UCS():
    def distance(p1, p2):
        return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])
    
    original_state_tuple = tuple(tuple(row) for row in original_state)
    que = []
    heapq.heappush(que, (0, original_state_tuple, []))  # (cost, state, positions)
    visited = {}

    while que:
        cost, c, positions = heapq.heappop(que)
        a = [list(row) for row in c]

        if a == goal_state:
            return a

        state_key = tuple(tuple(row) for row in a)
        if state_key in visited and visited[state_key] <= cost:
            continue
        visited[state_key] = cost

        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if a[i][j] == 1:
                    continue
                b = [row.copy() for row in a]
                b[i][j] = 1
                valid = True

                for r in range(BOARD_SIZE):
                    if sum(b[r]) > 1:
                        valid = False
                        break
                for c2 in range(BOARD_SIZE):
                    if sum(b[r][c2] for r in range(BOARD_SIZE)) > 1:
                        valid = False
                        break

                if not valid:
                    continue

                state = tuple(tuple(row) for row in b)

                if not positions:
                    step_cost = distance((0, 0), (i, j))
                else:
                    step_cost = distance(positions[-1], (i, j))

                new_cost = cost + (step_cost if step_cost > 0 else 1)
                new_positions = positions + [(i, j)]

                if state not in visited or new_cost < visited[state]:
                    heapq.heappush(que, (new_cost, state, new_positions))

def DLS(state=None, limit=4):
    if state is None:  
        state = tuple(tuple(row) for row in original_state)

    a = [list(row) for row in state]

    if a == goal_state:
        return a
    elif limit == 0:
        return "cutoff"
    else:
        cutoff_occurred = False
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                b = [row.copy() for row in a]
                if b[i][j] == 1:
                    continue
                b[i][j] = 1

                valid = True
                for r in range(BOARD_SIZE):
                    if sum(b[r]) > 1:
                        valid = False
                        break
                for c2 in range(BOARD_SIZE):
                    if sum(b[r][c2] for r in range(BOARD_SIZE)) > 1:
                        valid = False
                        break

                if not valid:
                    continue

                child = tuple(tuple(row) for row in b)

                result = DLS(child, limit - 1)

                if result == "cutoff":
                    cutoff_occurred = True
                elif result != "failure":
                    return result
                    
        if cutoff_occurred:
            return "cutoff"
        else:
            return "failure"

def IDS(max_depth = BOARD_SIZE):
    for depth in range(max_depth+1):
        result = DLS(limit=depth)
        if result != "cutoff" and result != "failure":
            return result
    return None
    
def GreedySearch():
    def heuristic(p1, goal):
        # Dùng Manhattan distance làm h(n)
        return abs(p1[0] - goal[0]) + abs(p1[1] - goal[1])
    
    original_state_tuple = tuple(tuple(row) for row in original_state)
    que = []
    heapq.heappush(que, (0, original_state_tuple, []))  # (h(n), state, positions)
    visited = set()

    while que:
        h_val, c, positions = heapq.heappop(que)
        a = [list(row) for row in c]

        if a == goal_state:
            return a

        state_key = tuple(tuple(row) for row in a)
        if state_key in visited:
            continue
        visited.add(state_key)

        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if a[i][j] == 1:
                    continue
                b = [row.copy() for row in a]
                b[i][j] = 1
                valid = True

                for r in range(BOARD_SIZE):
                    if sum(b[r]) > 1:
                        valid = False
                        break
                for c2 in range(BOARD_SIZE):
                    if sum(b[r][c2] for r in range(BOARD_SIZE)) > 1:
                        valid = False
                        break

                if not valid:
                    continue

                state = tuple(tuple(row) for row in b)
                new_positions = positions + [(i, j)]

                # h(n): khoảng cách Manhattan từ ô mới (i,j) tới goal queen cuối cùng
                # Giả sử goal cuối là vị trí (BOARD_SIZE-1, BOARD_SIZE-1) (theo goal_state được khai báo ở trên)
                h_n = heuristic((i, j), (BOARD_SIZE-1, BOARD_SIZE-1))
                heapq.heappush(que, (h_n, state, new_positions))
    return None

def A_star():
    def distance(p1, p2):
        # g(n) và h(n) đều dùng Manhattan distance trong ví dụ này
        return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

    original_state_tuple = tuple(tuple(row) for row in original_state)
    que = []
    # (f(n), g(n), state, positions)
    heapq.heappush(que, (0, 0, original_state_tuple, []))
    visited = {}

    while que:
        f_val, g_val, c, positions = heapq.heappop(que)
        a = [list(row) for row in c]

        if a == goal_state:
            return a

        state_key = tuple(tuple(row) for row in a)
        if state_key in visited and visited[state_key] <= g_val:
            continue
        visited[state_key] = g_val

        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if a[i][j] == 1:
                    continue
                b = [row.copy() for row in a]
                b[i][j] = 1
                valid = True

                for r in range(BOARD_SIZE):
                    if sum(b[r]) > 1:
                        valid = False
                        break
                for c2 in range(BOARD_SIZE):
                    if sum(b[r][c2] for r in range(BOARD_SIZE)) > 1:
                        valid = False
                        break

                if not valid:
                    continue

                state = tuple(tuple(row) for row in b)
                new_positions = positions + [(i, j)]

                # g(n): chi phí thật (dùng Manhattan giữa 2 quân liên tiếp)
                if not positions:
                    step_cost = distance((0, 0), (i, j))
                else:
                    step_cost = distance(positions[-1], (i, j))
                g_new = g_val + (step_cost if step_cost > 0 else 1)

                # h(n): ước lượng còn lại → ví dụ khoảng cách Manhattan
                # tới góc dưới phải (tùy định nghĩa goal)
                h_new = distance((i, j), (BOARD_SIZE-1, BOARD_SIZE-1))
                f_new = g_new + h_new

                heapq.heappush(que, (f_new, g_new, state, new_positions))
    return None

def Hill_Climbing(goal_state=goal_state, BOARD_SIZE=BOARD_SIZE):
    def conflicts(state):
        cnt = 0
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if state[i][j] == 1:
                    cnt += sum(state[i][k] for k in range(BOARD_SIZE)) - 1
                    cnt += sum(state[k][j] for k in range(BOARD_SIZE)) - 1
        return cnt // 2

    def random_state():
        state = [[0]*BOARD_SIZE for _ in range(BOARD_SIZE)]
        for i in range(BOARD_SIZE):
            j = random.randint(0, BOARD_SIZE-1)
            state[i][j] = 1
        return state

    current = random_state()
    current_cost = conflicts(current)
    history = [copy.deepcopy(current)]  # Lưu lại bước đầu tiên

    while True:
        neighbors = []
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if current[i][j] == 1:
                    for new_j in range(BOARD_SIZE):
                        if new_j != j:
                            new_state = [row[:] for row in current]
                            new_state[i][j] = 0
                            new_state[i][new_j] = 1
                            neighbors.append(new_state)

        next_state = None
        next_cost = float("inf")
        for n in neighbors:
            c = conflicts(n)
            if c < next_cost:
                next_cost = c
                next_state = n

        if next_cost >= current_cost:  # Local minima
            return current, history

        current = next_state
        current_cost = next_cost
        history.append(copy.deepcopy(current))

        if current == goal_state or current_cost == 0:
            return current, history

def Simulated_Annealing(goal_state=goal_state, BOARD_SIZE=BOARD_SIZE, max_iter=100000, T=100, cooling=0.99):
    def conflicts(state):
        cnt = 0
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if state[i][j] == 1:
                    cnt += sum(state[i][k] for k in range(BOARD_SIZE)) - 1
                    cnt += sum(state[k][j] for k in range(BOARD_SIZE)) - 1
        return cnt // 2

    def random_state():
        state = [[0]*BOARD_SIZE for _ in range(BOARD_SIZE)]
        for i in range(BOARD_SIZE):
            j = random.randint(0, BOARD_SIZE-1)
            state[i][j] = 1
        return state

    def random_neighbor(state):
        neighbor = [row[:] for row in state]
        row = random.randint(0, BOARD_SIZE-1)
        col_old = neighbor[row].index(1)
        col_new = random.randint(0, BOARD_SIZE-1)
        while col_new == col_old:
            col_new = random.randint(0, BOARD_SIZE-1)
        neighbor[row][col_old] = 0
        neighbor[row][col_new] = 1
        return neighbor

    current = random_state()
    current_cost = conflicts(current)
    history = [current]

    for step in range(max_iter):
        if current_cost == 0 or (goal_state is not None and current == goal_state):
            return current, history

        neighbor = random_neighbor(current)
        neighbor_cost = conflicts(neighbor)
        delta = neighbor_cost - current_cost

        if delta < 0 or random.random() < math.exp(-delta / T):
            current, current_cost = neighbor, neighbor_cost
            history.append(current)

        T *= cooling
        if T < 1e-6:
            break

    return current, history

def Genetic_Algorithm(target=goal_state, N=BOARD_SIZE, pop_size=100, mutation_rate=0.1, max_gen=1000):
    history = []  # lưu các cha mẹ từng thế hệ + con đích

    def random_state():
        state = [[0] * N for _ in range(N)]
        for i in range(N):
            c = random.randint(0, N - 1)
            state[i][c] = 1
        return state

    def fitness(state):
        cost = 0
        for i in range(N):
            col_state = state[i].index(1)
            col_target = target[i].index(1)
            cost += abs(col_state - col_target)
        return cost

    def selection(population):
        total_fit = sum(1 / (1 + fitness(ind)) for ind in population)
        pick = random.uniform(0, total_fit)
        current = 0
        for ind in population:
            current += 1 / (1 + fitness(ind))
            if current > pick:
                return ind
        return population[-1]

    def crossover(p1, p2):
        point = random.randint(1, N - 2)
        child1 = [row[:] for row in p1[:point]] + [row[:] for row in p2[point:]]
        child2 = [row[:] for row in p2[:point]] + [row[:] for row in p1[point:]]
        return child1, child2

    def mutate(state):
        if random.random() < mutation_rate:
            r = random.randint(0, N - 1)
            c_old = state[r].index(1)
            c_new = random.randint(0, N - 1)
            while c_new == c_old:
                c_new = random.randint(0, N - 1)
            state[r][c_old] = 0
            state[r][c_new] = 1
        return state

    # --- main loop ---
    population = [random_state() for _ in range(pop_size)]
    best = min(population, key=fitness)

    for _ in range(max_gen):
        if best == target:
            history.append(best)  # thêm con đích vào lịch sử
            return best, history

        new_population = []
        parents_this_gen = []  # lưu cha mẹ của thế hệ này
        while len(new_population) < pop_size:
            p1 = selection(population)
            p2 = selection(population)
            parents_this_gen.extend([p1, p2])
            c1, c2 = crossover(p1, p2)
            new_population.append(mutate(c1))
            new_population.append(mutate(c2))

        history.append(parents_this_gen)  # lưu cha mẹ của thế hệ
        population = new_population
        best = min(population, key=fitness)

    history.append(best)  # nếu kết thúc mà chưa đạt target, lưu best
    return best, history


def Beam_Search(k=2):
    # ----- TẤT CẢ LOGIC TÌM KIẾM CỦA BẠN ĐƯỢC GIỮ NGUYÊN -----
    history_state = []  # Vẫn lưu dạng [1, 3, 0] như cũ

    def cost_function(state):
        conflicts = 0
        for i in range(len(state)):
            for j in range(i + 1, len(state)):
                if state[i] == state[j] or abs(state[i] - state[j]) == j - i:
                    conflicts += 1
        return conflicts

    beam = [(0, [])]

    for row in range(BOARD_SIZE):
        candidates = []
        for cost, state in beam:
            for col in range(BOARD_SIZE):
                if col not in state:
                    new_state = state + [col]
                    new_cost = cost_function(new_state)
                    candidates.append((new_cost, new_state))

        if not candidates:
            # --- THAY ĐỔI 1: Định dạng lại history_state trước khi return ---
            reformatted_history = []
            for state in history_state:
                board_step = [[0] * BOARD_SIZE for _ in range(BOARD_SIZE)]
                for r, c in enumerate(state):
                    board_step[r][c] = 1
                reformatted_history.append(board_step)
            return None, reformatted_history

        beam = heapq.nsmallest(k, candidates, key=lambda x: x[0])
        best_cost, best_state = min(beam, key=lambda x: x[0])
        history_state.append(best_state) # Vẫn append state [1, 3, 0] như cũ

        if len(best_state) == BOARD_SIZE and best_cost == 0:
            board = [[0] * BOARD_SIZE for _ in range(BOARD_SIZE)]
            for r, c in enumerate(best_state):
                board[r][c] = 1
            
            # --- THAY ĐỔI 2: Định dạng lại history_state trước khi return ---
            reformatted_history = []
            for state in history_state:
                board_step = [[0] * BOARD_SIZE for _ in range(BOARD_SIZE)]
                for r, c in enumerate(state):
                    board_step[r][c] = 1
                reformatted_history.append(board_step)
            return board, reformatted_history

    # ----- LOGIC KẾT THÚC CỦA BẠN ĐƯỢC GIỮ NGUYÊN -----
    best_cost, best_state = min(beam, key=lambda x: x[0])
    board = [[0] * BOARD_SIZE for _ in range(BOARD_SIZE)]
    for r, c in enumerate(best_state):
        board[r][c] = 1

    # --- THAY ĐỔI 3: Định dạng lại history_state lần cuối trước khi return ---
    reformatted_history = []
    for state in history_state:
        board_step = [[0] * BOARD_SIZE for _ in range(BOARD_SIZE)]
        for r, c in enumerate(state):
            board_step[r][c] = 1
        reformatted_history.append(board_step)
        
    return board, reformatted_history
    
def DFS_AND_OR():
    goal_state_tuple = tuple(tuple(row) for row in goal_state)

    # --- Hàm đệ quy nội tuyến thực hiện logic tìm kiếm cốt lõi ---
    def _search(state_tuple, path_history):
        # Nếu là trạng thái đích -> trả về đường đi chỉ chứa chính nó
        if state_tuple == goal_state_tuple:
            return [state_tuple]

        # Nếu gặp chu trình -> thất bại
        if state_tuple in path_history:
            return None

        # Logic OR: Duyệt qua mọi hành động có thể
        current_board = [list(row) for row in state_tuple]
        for r in range(BOARD_SIZE):
            for c in range(BOARD_SIZE):
                if current_board[r][c] == 0:
                    temp_board = [row[:] for row in current_board]
                    temp_board[r][c] = 1
                    is_valid_action = True
                    if sum(temp_board[r]) > 1: is_valid_action = False
                    if is_valid_action and sum(temp_board[i][c] for i in range(BOARD_SIZE)) > 1: is_valid_action = False
                    
                    if is_valid_action:
                        # Logic AND: Mọi kết quả của hành động phải giải được
                        result_state = tuple(tuple(row) for row in temp_board)
                        path_from_child = _search(result_state, path_history + [state_tuple])
                        
                        if path_from_child is not None:
                            full_path = [state_tuple] + path_from_child
                            return full_path
        
        # Nếu không có hành động nào thành công -> thất bại
        return None

    # --- Phần thân chính của hàm ---
    initial_state_tuple = tuple(tuple(row) for row in original_state)
    final_path_tuples = _search(initial_state_tuple, [])
    
    if final_path_tuples is not None:
        # 1. Bảng kết quả cuối cùng
        final_board = [list(row) for row in final_path_tuples[-1]]
        
        # 2. Lịch sử đường đi (list các bảng)
        history_state = [[list(row) for row in state] for state in final_path_tuples]
        
        return final_board, history_state
    else:
        # Trả về None và một list rỗng nếu không tìm thấy
        return None, []
        

def Belief_Search():
    """
    Sử dụng BFS để tìm một KẾ HOẠCH HÀNH ĐỘNG (chuỗi các lần quét)
    để xác định vị trí của một quân xe bị giấu, dựa trên các ghi chú của bạn.
    """
    # --- Các hàm nội tuyến ---
    def _update_belief(belief, ping_action, observation):
        new_belief = set()
        action_type, index = ping_action
        for r, c in belief:
            pos_is_consistent = ((r == index) if action_type == 'scan_row' else (c == index)) == observation
            if pos_is_consistent:
                new_belief.add((r, c))
        return frozenset(new_belief)

    # --- Phần thân chính của BFS ---

    # 1. Trạng thái "niềm tin ban đầu" là một tập hợp các trạng thái có thể có
    initial_belief = frozenset((r, c) for r in range(BOARD_SIZE) for c in range(BOARD_SIZE))
    
    # 2. Hàng đợi `open` chứa node đầu tiên (niềm tin ban đầu)
    queue = collections.deque([(initial_belief, [])])
    visited = {initial_belief}

    # 3. "action": hợp các hành động quét
    all_possible_actions = [('scan_row', i) for i in range(BOARD_SIZE)] + \
                           [('scan_col', i) for i in range(BOARD_SIZE)]

    # 4. Vòng lặp tìm kiếm
    while queue:
        current_belief, plan = queue.popleft()

        # 5. Kiểm tra "trạng thái niềm tin kết thúc"
        if len(current_belief) == 1:
            found_pos = list(current_belief)[0]
            final_board = [[0] * BOARD_SIZE for _ in range(BOARD_SIZE)]
            final_board[found_pos[0]][found_pos[1]] = 1
            return final_board, plan

        # 6. Mở rộng node, tạo ra các trạng thái con
        for action in all_possible_actions:
            # Agent phải xem xét mọi kết quả quan sát có thể có (True hoặc False)
            for observation in [True, False]:
                next_belief = _update_belief(current_belief, action, observation)
                
                if next_belief and next_belief not in visited:
                    visited.add(next_belief)
                    new_plan = plan + [action]
                    queue.append((next_belief, new_plan))
                    
    return None, []
def draw_board(canvas, with_cars=False):
    global timer_running
    canvas.delete("all")
    for row in range(BOARD_SIZE):
        for col in range(BOARD_SIZE):
            x1, y1 = col * CELL_SIZE, row * CELL_SIZE
            x2, y2 = x1 + CELL_SIZE, y1 + CELL_SIZE
            color = "white" if (row + col) % 2 == 0 else "gray"
            canvas.create_rectangle(x1, y1, x2, y2,
                                    fill=color, outline="black")

    if with_cars and cars_positions:
        coords = [(r, c) for r in range(BOARD_SIZE)
                        for c in range(BOARD_SIZE)
                        if cars_positions[r][c] == 1]

        def draw_next(i=0):
            if i < len(coords):
                r, c = coords[i]
                x1, y1 = c * CELL_SIZE, r * CELL_SIZE
                canvas.create_text(x1 + CELL_SIZE/2,
                                   y1 + CELL_SIZE/2,
                                   text="♖",
                                   font=("Arial", 28),
                                   fill="red")
                canvas.after(1000, lambda: draw_next(i+1))
        draw_next()

def show_algorithm_code(event):
    global algorithm_frame, algorithm_combobox, selected
    for widget in algorithm_frame.winfo_children():
        widget.destroy()
    selected = algorithm_combobox.get()
    if selected == "Uninformed Search             ---   Breadth First Search":
        code_text = """def BFS():
    node = tuple(tuple(row) for row in original_state)  
    if node == tuple(tuple(row) for row in goal_state):  
        return goal_state

    frontier = deque([node]) 
    explored = set()          

    while frontier:
        node = frontier.popleft()   
        explored.add(node)

        a = [list(row) for row in node]  
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                b = [list(row) for row in a]
                if b[i][j] == 1:
                    continue
                b[i][j] = 1

                valid = True
                for r in range(BOARD_SIZE):
                    if sum(b[r]) > 1:
                        valid = False
                        break
                if valid:
                    for c2 in range(BOARD_SIZE):
                        if sum(b[r][c2] for r in range(BOARD_SIZE)) > 1:
                            valid = False
                            break

                if valid:
                    child = tuple(tuple(row) for row in b)
                    if child not in explored and all(child != n for n in frontier):
                        if child == tuple(tuple(row) for row in goal_state):
                            return [list(row) for row in child]
                        frontier.append(child)
    return None"""
        
    elif selected == "Uninformed Search             ---   Depth First Search":
        code_text = """def DFS():
    node = tuple(tuple(row) for row in original_state)  
    if node == tuple(tuple(row) for row in goal_state):  
        return goal_state

    stack = [node]        
    visited = {node}    

    while stack:
        node = stack.pop()    
        a = [list(row) for row in node]

        if a == goal_state:
            return a

        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                b = [list(row) for row in a]
                if b[i][j] == 1:
                    continue
                b[i][j] = 1

                valid = True
                for r in range(BOARD_SIZE):
                    if sum(b[r]) > 1:
                        valid = False
                        break
                if valid:
                    for c2 in range(BOARD_SIZE):
                        if sum(b[r][c2] for r in range(BOARD_SIZE)) > 1:
                            valid = False
                            break

                if valid:
                    state = tuple(tuple(row) for row in b)
                    if state not in visited:
                        visited.add(state)
                        stack.append(state)  
    return None"""
        
    elif selected == "Uninformed Search             ---   Depth Limited Search":
        code_text = """ def DLS(state=None, limit=4):
    if state is None:  
        state = tuple(tuple(row) for row in original_state)

    a = [list(row) for row in state]

    if a == goal_state:
        return a
    elif limit == 0:
        return "cutoff"
    else:
        cutoff_occurred = False
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                b = [row.copy() for row in a]
                if b[i][j] == 1:
                    continue
                b[i][j] = 1

                valid = True
                for r in range(BOARD_SIZE):
                    if sum(b[r]) > 1:
                        valid = False
                        break
                for c2 in range(BOARD_SIZE):
                    if sum(b[r][c2] for r in range(BOARD_SIZE)) > 1:
                        valid = False
                        break

                if not valid:
                    continue

                child = tuple(tuple(row) for row in b)

                result = DLS(child, limit - 1)

                if result == "cutoff":
                    cutoff_occurred = True
                elif result != "failure":
                    return result
                    
        if cutoff_occurred:
            return "cutoff"
        else:
            return "failure" """
        
    elif selected == "Uninformed Search             ---   Uniform Cost Search":
        code_text = """def UCS():
    def distance(p1, p2):
        return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])
    
    original_state_tuple = tuple(tuple(row) for row in original_state)
    que = []
    heapq.heappush(que, (0, original_state_tuple, []))  # (cost, state, positions)
    visited = {}

    while que:
        cost, c, positions = heapq.heappop(que)
        a = [list(row) for row in c]

        if a == goal_state:
            return a

        state_key = tuple(tuple(row) for row in a)
        if state_key in visited and visited[state_key] <= cost:
            continue
        visited[state_key] = cost

        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if a[i][j] == 1:
                    continue
                b = [row.copy() for row in a]
                b[i][j] = 1
                valid = True

                for r in range(BOARD_SIZE):
                    if sum(b[r]) > 1:
                        valid = False
                        break
                for c2 in range(BOARD_SIZE):
                    if sum(b[r][c2] for r in range(BOARD_SIZE)) > 1:
                        valid = False
                        break

                if not valid:
                    continue

                state = tuple(tuple(row) for row in b)

                if not positions:
                    step_cost = distance((0, 0), (i, j))
                else:
                    step_cost = distance(positions[-1], (i, j))

                new_cost = cost + (step_cost if step_cost > 0 else 1)
                new_positions = positions + [(i, j)]

                if state not in visited or new_cost < visited[state]:
                    heapq.heappush(que, (new_cost, state, new_positions))"""
        
    elif selected == "Uninformed Search             ---   Iterative Deepening Search":
        code_text = """def IDS(max_depth = BOARD_SIZE):
    for depth in range(max_depth+1):
        result = DLS(limit=depth)
        if result != "cutoff" and result != "failure":
            return result
    return None"""
        
    elif selected == "Informed Search        ---   Greedy Search":
        code_text = """def GreedySearch():
    def heuristic(p1, goal):
        # Dùng Manhattan distance làm h(n)
        return abs(p1[0] - goal[0]) + abs(p1[1] - goal[1])
    
    original_state_tuple = tuple(tuple(row) for row in original_state)
    que = []
    heapq.heappush(que, (0, original_state_tuple, []))  # (h(n), state, positions)
    visited = set()

    while que:
        h_val, c, positions = heapq.heappop(que)
        a = [list(row) for row in c]

        if a == goal_state:
            return a

        state_key = tuple(tuple(row) for row in a)
        if state_key in visited:
            continue
        visited.add(state_key)

        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if a[i][j] == 1:
                    continue
                b = [row.copy() for row in a]
                b[i][j] = 1
                valid = True

                for r in range(BOARD_SIZE):
                    if sum(b[r]) > 1:
                        valid = False
                        break
                for c2 in range(BOARD_SIZE):
                    if sum(b[r][c2] for r in range(BOARD_SIZE)) > 1:
                        valid = False
                        break

                if not valid:
                    continue

                state = tuple(tuple(row) for row in b)
                new_positions = positions + [(i, j)]

                # h(n): khoảng cách Manhattan từ ô mới (i,j) tới goal queen cuối cùng
                # Giả sử goal cuối là vị trí (BOARD_SIZE-1, BOARD_SIZE-1) (theo goal_state được khai báo ở trên)
                h_n = heuristic((i, j), (BOARD_SIZE-1, BOARD_SIZE-1))
                heapq.heappush(que, (h_n, state, new_positions))
    return None"""

    elif selected == "Informed Search        ---   A_star":
        code_text = """def A_star():
    def distance(p1, p2):
        # g(n) và h(n) đều dùng Manhattan distance trong ví dụ này
        return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

    original_state_tuple = tuple(tuple(row) for row in original_state)
    que = []
    # (f(n), g(n), state, positions)
    heapq.heappush(que, (0, 0, original_state_tuple, []))
    visited = {}

    while que:
        f_val, g_val, c, positions = heapq.heappop(que)
        a = [list(row) for row in c]

        if a == goal_state:
            return a

        state_key = tuple(tuple(row) for row in a)
        if state_key in visited and visited[state_key] <= g_val:
            continue
        visited[state_key] = g_val

        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if a[i][j] == 1:
                    continue
                b = [row.copy() for row in a]
                b[i][j] = 1
                valid = True

                for r in range(BOARD_SIZE):
                    if sum(b[r]) > 1:
                        valid = False
                        break
                for c2 in range(BOARD_SIZE):
                    if sum(b[r][c2] for r in range(BOARD_SIZE)) > 1:
                        valid = False
                        break

                if not valid:
                    continue

                state = tuple(tuple(row) for row in b)
                new_positions = positions + [(i, j)]

                # g(n): chi phí thật (dùng Manhattan giữa 2 quân liên tiếp)
                if not positions:
                    step_cost = distance((0, 0), (i, j))
                else:
                    step_cost = distance(positions[-1], (i, j))
                g_new = g_val + (step_cost if step_cost > 0 else 1)

                # h(n): ước lượng còn lại → ví dụ khoảng cách Manhattan
                # tới góc dưới phải (tùy định nghĩa goal)
                h_new = distance((i, j), (BOARD_SIZE-1, BOARD_SIZE-1))
                f_new = g_new + h_new

                heapq.heappush(que, (f_new, g_new, state, new_positions))
    return None"""

    elif selected == "Local Search                         ---   Hill_Climbing":
        code_text = """def Hill_Climbing(goal_state=goal_state, BOARD_SIZE=BOARD_SIZE):
    def conflicts(state):
        cnt = 0
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if state[i][j] == 1:
                    cnt += sum(state[i][k] for k in range(BOARD_SIZE)) - 1
                    cnt += sum(state[k][j] for k in range(BOARD_SIZE)) - 1
        return cnt // 2

    def random_state():
        state = [[0]*BOARD_SIZE for _ in range(BOARD_SIZE)]
        for i in range(BOARD_SIZE):
            j = random.randint(0, BOARD_SIZE-1)
            state[i][j] = 1
        return state

    current = random_state()
    current_cost = conflicts(current)
    history = [copy.deepcopy(current)]  # Lưu lại bước đầu tiên

    while True:
        neighbors = []
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if current[i][j] == 1:
                    for new_j in range(BOARD_SIZE):
                        if new_j != j:
                            new_state = [row[:] for row in current]
                            new_state[i][j] = 0
                            new_state[i][new_j] = 1
                            neighbors.append(new_state)

        next_state = None
        next_cost = float("inf")
        for n in neighbors:
            c = conflicts(n)
            if c < next_cost:
                next_cost = c
                next_state = n

        if next_cost >= current_cost:  # Local minima
            return current, history

        current = next_state
        current_cost = next_cost
        history.append(copy.deepcopy(current))

        if current == goal_state or current_cost == 0:
            return current, history"""

    elif selected == "Local Search                         ---   Simulated_Annealing":
        code_text = """def Simulated_Annealing(goal_state=goal_state, BOARD_SIZE=BOARD_SIZE, max_iter=100000, T=100, cooling=0.99):
    def conflicts(state):
        cnt = 0
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if state[i][j] == 1:
                    cnt += sum(state[i][k] for k in range(BOARD_SIZE)) - 1
                    cnt += sum(state[k][j] for k in range(BOARD_SIZE)) - 1
        return cnt // 2

    def random_state():
        state = [[0]*BOARD_SIZE for _ in range(BOARD_SIZE)]
        for i in range(BOARD_SIZE):
            j = random.randint(0, BOARD_SIZE-1)
            state[i][j] = 1
        return state

    def random_neighbor(state):
        neighbor = [row[:] for row in state]
        row = random.randint(0, BOARD_SIZE-1)
        col_old = neighbor[row].index(1)
        col_new = random.randint(0, BOARD_SIZE-1)
        while col_new == col_old:
            col_new = random.randint(0, BOARD_SIZE-1)
        neighbor[row][col_old] = 0
        neighbor[row][col_new] = 1
        return neighbor

    current = random_state()
    current_cost = conflicts(current)
    history = [current]

    for step in range(max_iter):
        if current_cost == 0 or (goal_state is not None and current == goal_state):
            return current, history

        neighbor = random_neighbor(current)
        neighbor_cost = conflicts(neighbor)
        delta = neighbor_cost - current_cost

        if delta < 0 or random.random() < math.exp(-delta / T):
            current, current_cost = neighbor, neighbor_cost
            history.append(current)

        T *= cooling
        if T < 1e-6:
            break

    return current, history"""

    elif selected == "Local Search                         ---   Genetic_Algorithm":
        code_text = """def Genetic_Algorithm(target=goal_state, N=BOARD_SIZE, pop_size=100, mutation_rate=0.1, max_gen=1000):
    history = []  # lưu các cha mẹ từng thế hệ + con đích

    def random_state():
        state = [[0] * N for _ in range(N)]
        for i in range(N):
            c = random.randint(0, N - 1)
            state[i][c] = 1
        return state

    def fitness(state):
        cost = 0
        for i in range(N):
            col_state = state[i].index(1)
            col_target = target[i].index(1)
            cost += abs(col_state - col_target)
        return cost

    def selection(population):
        total_fit = sum(1 / (1 + fitness(ind)) for ind in population)
        pick = random.uniform(0, total_fit)
        current = 0
        for ind in population:
            current += 1 / (1 + fitness(ind))
            if current > pick:
                return ind
        return population[-1]

    def crossover(p1, p2):
        point = random.randint(1, N - 2)
        child1 = [row[:] for row in p1[:point]] + [row[:] for row in p2[point:]]
        child2 = [row[:] for row in p2[:point]] + [row[:] for row in p1[point:]]
        return child1, child2

    def mutate(state):
        if random.random() < mutation_rate:
            r = random.randint(0, N - 1)
            c_old = state[r].index(1)
            c_new = random.randint(0, N - 1)
            while c_new == c_old:
                c_new = random.randint(0, N - 1)
            state[r][c_old] = 0
            state[r][c_new] = 1
        return state

    # --- main loop ---
    population = [random_state() for _ in range(pop_size)]
    best = min(population, key=fitness)

    for _ in range(max_gen):
        if best == target:
            history.append(best)  # thêm con đích vào lịch sử
            return best, history

        new_population = []
        parents_this_gen = []  # lưu cha mẹ của thế hệ này
        while len(new_population) < pop_size:
            p1 = selection(population)
            p2 = selection(population)
            parents_this_gen.extend([p1, p2])
            c1, c2 = crossover(p1, p2)
            new_population.append(mutate(c1))
            new_population.append(mutate(c2))

        history.append(parents_this_gen)  # lưu cha mẹ của thế hệ
        population = new_population
        best = min(population, key=fitness)

    history.append(best)  # nếu kết thúc mà chưa đạt target, lưu best
    return best, history"""

    elif selected == "Local Search                         ---   Beam_Search":
        code_text = """def Beam_Search(k=2):
    history_state = []  # lưu đường đi duy nhất

    def cost_function(state):
        conflicts = 0
        for i in range(len(state)):
            for j in range(i + 1, len(state)):
                if state[i] == state[j] or abs(state[i] - state[j]) == j - i:
                    conflicts += 1
        return conflicts

    beam = [(0, [])]

    for row in range(BOARD_SIZE):
        candidates = []
        for cost, state in beam:
            for col in range(BOARD_SIZE):
                if col not in state:
                    new_state = state + [col]
                    new_cost = cost_function(new_state)
                    candidates.append((new_cost, new_state))

        if not candidates:
            return None, history_state

        # Chọn k best, nhưng lưu **một state dẫn đến board tốt nhất hiện tại**
        beam = heapq.nsmallest(k, candidates, key=lambda x: x[0])
        # Chọn state tốt nhất trong beam làm bước tiếp theo của đường đi
        best_cost, best_state = min(beam, key=lambda x: x[0])
        history_state.append(best_state)

        # Nếu đã đạt goal
        if len(best_state) == BOARD_SIZE:
            board = [[0] * BOARD_SIZE for _ in range(BOARD_SIZE)]
            for r, c in enumerate(best_state):
                board[r][c] = 1
            return board, history_state

    # Nếu hết vòng lặp mà chưa đạt goal → trả best
    best_cost, best_state = min(beam, key=lambda x: x[0])
    board = [[0] * BOARD_SIZE for _ in range(BOARD_SIZE)]
    for r, c in enumerate(best_state):
        board[r][c] = 1
    return board, history_state"""
    elif selected == "Nondeterministic Search    ---   DFS_AND_OR":
        code_text = """    
def DFS_AND_OR():
    goal_state_tuple = tuple(tuple(row) for row in goal_state)

    # --- Hàm đệ quy nội tuyến thực hiện logic tìm kiếm cốt lõi ---
    def _search(state_tuple, path_history):
        # Nếu là trạng thái đích -> trả về đường đi chỉ chứa chính nó
        if state_tuple == goal_state_tuple:
            return [state_tuple]

        # Nếu gặp chu trình -> thất bại
        if state_tuple in path_history:
            return None

        # Logic OR: Duyệt qua mọi hành động có thể
        current_board = [list(row) for row in state_tuple]
        for r in range(BOARD_SIZE):
            for c in range(BOARD_SIZE):
                if current_board[r][c] == 0:
                    temp_board = [row[:] for row in current_board]
                    temp_board[r][c] = 1
                    is_valid_action = True
                    if sum(temp_board[r]) > 1: is_valid_action = False
                    if is_valid_action and sum(temp_board[i][c] for i in range(BOARD_SIZE)) > 1: is_valid_action = False
                    
                    if is_valid_action:
                        # Logic AND: Mọi kết quả của hành động phải giải được
                        result_state = tuple(tuple(row) for row in temp_board)
                        path_from_child = _search(result_state, path_history + [state_tuple])
                        
                        if path_from_child is not None:
                            full_path = [state_tuple] + path_from_child
                            return full_path
        
        # Nếu không có hành động nào thành công -> thất bại
        return None

    # --- Phần thân chính của hàm ---
    initial_state_tuple = tuple(tuple(row) for row in original_state)
    final_path_tuples = _search(initial_state_tuple, [])
    
    if final_path_tuples is not None:
        # 1. Bảng kết quả cuối cùng
        final_board = [list(row) for row in final_path_tuples[-1]]
        
        # 2. Lịch sử đường đi (list các bảng)
        history_state = [[list(row) for row in state] for state in final_path_tuples]
        
        return final_board, history_state
    else:
        # Trả về None và một list rỗng nếu không tìm thấy
        return None, []"""
    else:
        code_text = "# Chưa có code"
        
    # Xóa widget cũ
    for widget in algorithm_frame.winfo_children():
        widget.destroy()
    
    # Tạo Text widget với scrollbar
    code_widget = tk.Text(algorithm_frame, font=("Consolas", 12), wrap="none", bg="white")
    code_widget.insert("1.0", code_text)
    code_widget.config(state="disabled")  
    code_widget.pack(side="left", fill="both", expand=True)
    
    scrollbar = tk.Scrollbar(algorithm_frame, command=code_widget.yview)
    scrollbar.pack(side="right", fill="y")
    code_widget.config(yscrollcommand=scrollbar.set)

def show_total_state(history_state):
    global state_frame
    for widget in state_frame.winfo_children():
        widget.destroy()

    formatted_text = re.sub(r"\](?!$)", "]\n", history_state)

    code_widget = tk.Text(state_frame, font=("Consolas", 12,"bold"), wrap="char", bg="white")
    code_widget.insert("1.0", formatted_text)  # dùng formatted_text thay vì history_state
    code_widget.config(state="disabled")  
    code_widget.pack(side="left", fill="both", expand=True)
    
    scrollbar = tk.Scrollbar(state_frame, command=code_widget.yview)
    scrollbar.pack(side="right", fill="y")
    code_widget.config(yscrollcommand=scrollbar.set)
    
def start_algo():
    global cars_positions, canvas_right_global, timer_running, timer_elapse
    timer_elapse = 0
    timer_running = True
    update_timer()
    if selected == "Uninformed Search             ---   Breadth First Search":
        cars_positions = BFS()
        history_state = "⚠ Vui lòng chọn 1 thuật toán trong các\n"\
                        "nhóm thuật toán: Local Search để xem\n"\
                        "được lịch sử các trạng thái đã đi qua ⚠"      
    elif selected == "Uninformed Search             ---   Depth First Search":
        cars_positions = DFS()
        history_state = "⚠ Vui lòng chọn 1 thuật toán trong các\n"\
                        "nhóm thuật toán: Local Search để xem\n"\
                        "được lịch sử các trạng thái đã đi qua ⚠"      
    elif selected == "Uninformed Search             ---   Uniform Cost Search":
        cars_positions = UCS()
        history_state = "⚠ Vui lòng chọn 1 thuật toán trong các\n"\
                        "nhóm thuật toán: Local Search để xem\n"\
                        "được lịch sử các trạng thái đã đi qua ⚠"      
    elif selected == "Uninformed Search             ---   Depth Limited Search":
        cars_positions = DLS()
        history_state = "⚠ Vui lòng chọn 1 thuật toán trong các\n"\
                        "nhóm thuật toán: Local Search để xem\n"\
                        "được lịch sử các trạng thái đã đi qua ⚠"
    elif selected == "Uninformed Search             ---   Iterative Deepening Search":
        cars_positions = IDS()
        history_state = "⚠ Vui lòng chọn 1 thuật toán trong các\n"\
                        "nhóm thuật toán: Local Search để xem\n"\
                        "được lịch sử các trạng thái đã đi qua ⚠"
    elif selected == "Informed Search                  ---   Greedy Search":
        cars_positions = GreedySearch()
        history_state = "⚠ Vui lòng chọn 1 thuật toán trong các\n"\
                        "nhóm thuật toán: Local Search để xem\n"\
                        "được lịch sử các trạng thái đã đi qua ⚠"
    elif selected == "Informed Search                  ---   A_star":
        cars_positions = A_star()
        history_state = "⚠ Vui lòng chọn 1 thuật toán trong các\n"\
                        "nhóm thuật toán: Local Search để xem\n"\
                        "được lịch sử các trạng thái đã đi qua ⚠"
    elif selected == "Local Search                         ---   Hill_Climbing":
        cars_positions, history = Hill_Climbing() 
        history_state = "💡 Lịch sử các trạng thái bàn cờ đã đi qua:\n\n".format(len(history))
        history_state += str(history)  
    elif selected == "Local Search                         ---   Simulated_Annealing":
        cars_positions, history = Simulated_Annealing() 
        history_state = "💡 Lịch sử các trạng thái bàn cờ đã đi qua:\n\n".format(len(history))
        history_state += str(history)  
    elif selected == "Local Search                         ---   Genetic_Algorithm":
        cars_positions, history = Genetic_Algorithm() 
        history_state = "Lịch sử các trạng thái bàn cờ đã đi qua:\n\n"\
                        "💡 Ma trận đầu tiên: Random\n"\
                        "💡 Ma trận cuối: Trạng thái đích\n"\
                        "💡 Các ma trận còn lại lần lượt là các cặp\n"\
                        "CHA và MẸ của từng thế hệ để tạo\n"\
                        "quần thể mới:\n\n".format(len(history))
        history_state += str(history)  
    elif selected == "Local Search                         ---   Beam_Search":
        cars_positions, history = Beam_Search() 
        history_state = "💡 Lịch sử các trạng thái bàn cờ đã đi qua:\n\n".format(len(history))
        history_state += str(history)   
    elif selected == "Nondeterministic Search    ---   DFS_AND_OR":
        cars_positions, history = DFS_AND_OR()
        history_state = "💡 Lịch sử các trạng thái bàn cờ đã đi qua:\n\n".format(len(history))
        history_state += str(history)
    elif selected == "Search_with_Belief":
        cars_positions, history = Belief_Search()
        history_state = "💡 Lịch sử các trạng thái bàn cờ đã đi qua:\n\n".format(len(history))
        history_state += str(history)
    else:
        cars_positions = None
  
    draw_board(canvas_right_global, with_cars=True)
    show_total_state(history_state)
    timer_running = False
    
def main():
    root=tk.Tk()
    root.title("Bài toán N quân xe")
    root.configure(bg="#ecf0f1")
    root.state("zoomed")
    root.resizable(False,False)

    title_label = tk.Label(root,
                           text="BÀI TOÁN N QUÂN XE",
                           font=("Arial",35,"bold"),
                           bg="#ecf0f1",
                           fg="#2c3e50")
    title_label.pack(side="top",pady=20) 

    # Left frame
    left_frame = tk.Frame(root, bg="#34495e")
    left_frame.place(relx=0.0, rely=0.0, anchor="nw",
                        x=80, y=180, width=800, height=830)
    
    global algorithm_combobox, algorithm_frame, state_frame
    algorithm_combobox= ttk.Combobox(left_frame, 
                                  values=["Uninformed Search             ---   Breadth First Search", 
                                          "Uninformed Search             ---   Depth First Search", 
                                          "Uninformed Search             ---   Uniform Cost Search", 
                                          "Uninformed Search             ---   Depth Limited Search",
                                          "Uninformed Search             ---   Iterative Deepening Search", 
                                          "Informed Search                  ---   Greedy Search",
                                          "Informed Search                  ---   A_star",
                                          "Local Search                         ---   Hill_Climbing",
                                          "Local Search                         ---   Simulated_Annealing",
                                          "Local Search                         ---   Genetic_Algorithm",
                                          "Local Search                         ---   Beam_Search",
                                          "Nondeterministic Search    ---   DFS_AND_OR",
                                         "Search_with_Belief"],
                                  font=("Arial", 13, "bold"))
    algorithm_combobox.current(0)  
    algorithm_combobox.pack(side="top", fill="x", padx=10, pady=10)
    algorithm_combobox.bind("<<ComboboxSelected>>", show_algorithm_code)

    algorithm_frame = tk.Frame(left_frame, bg="white")
    algorithm_frame.pack(fill="both", expand=True, padx=15, pady=20)

    # State_frame
    state_frame = tk.Frame(root, bg="white", width=400, height=300)
    state_frame.pack(side="top", anchor="ne", padx=125, pady=75, expand=True)
    state_frame.pack_propagate(False)  
    state_label = tk.Label(state_frame, text="Lịch sử trạng thái\n"\
                           "của nhóm thuật toán \n"\
                           "Local Search", font=("Arial",30,"bold"),bg="#34495e",fg="white")
    state_label.pack(expand=True) 

    # Control frame
    control_frame = tk.Frame(root, bg="#34495e")
    control_frame.place(relx=1.0, rely=1.0, anchor="se",
                        y=-BOARD_SIZE*CELL_SIZE-120,
                        x=-BOARD_SIZE*CELL_SIZE+100)

    global timer_label
    timer_label = tk.Label(control_frame, text="Thời gian: 0 s",
                           font=("Arial",18))
    timer_label.grid(row=0, column=1, columnspan=4, pady=15)

    btn_start = tk.Button(control_frame, text="Bắt đầu", font=("Arial",18), command=start_algo)
    btn_start.grid(row=1, column=1, padx=15, pady=15)

    btn_reset = tk.Button(control_frame, text="Chơi lại", font=("Arial",18), command=reset_game)
    btn_reset.grid(row=1, column=3, padx=15, pady=15)
        
    # Board frame
    frame = tk.Frame(root, bg="black")
    frame.place(relx=1.0, rely=1.0, anchor="se", y=-80, x=-80)

    canvas_left = tk.Canvas(frame,
                            width= BOARD_SIZE * CELL_SIZE,
                            height= BOARD_SIZE * CELL_SIZE)
    canvas_left.grid(row=0, column=0, padx=5, pady=5)
    draw_board(canvas_left, with_cars=False)

    canvas_right = tk.Canvas(frame,
                             width = BOARD_SIZE * CELL_SIZE,
                             height = BOARD_SIZE * CELL_SIZE)
    canvas_right.grid(row=0, column=1, padx=5, pady=5)
    global canvas_right_global
    canvas_right_global = canvas_right
    draw_board(canvas_right_global, with_cars=False)

    root.mainloop()
    
if __name__ == "__main__":
    main()
 
 
 